In [ ]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Reel
using Revise
using EmergencyBrakingSystem

using CSV
using DataFrames

In [ ]:
### Evaluate all EuroNCAP scenarios

# Definition EuroNCAP scenarios, ego vehicle and pedestrian behavior
scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA", "FP"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6

results = Vector[]
for scenario in scenarios
    println("Scenario: ", scenario)
    if (scenario == "FP") 
        hit_points = [-100., 200.]
    else
        hit_points = [0., 10., 20., 30., 40., 50.]
    end
    for hit_point in hit_points
        for vut_speed in vut_speeds
            ego_v = vut_speed
            (ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = EmergencyBrakingSystem.generate_scenario(scenario, ego_v, hit_point)
            
            (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
            (collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = EmergencyBrakingSystem.evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision, ped_x)
            println("Collision: ", collision, " eb: ", emergency_brake_intervention, " ", round(dv_collision, digits=1), " ", round(v_mean, digits=1), " ", round(a_mean, digits=1), " ", a_jerk, " ", a_min)
            result = [scenario_id, ego_v, hit_point, collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min  ]
            push!(results, result)
        end
    end 
end  


# store results from scneario evaluation in log file
df = DataFrame(results)
df = DataFrame(Matrix(df)')
rename!(df, :x1 => :scenario_id, :x2 => :ego_v, :x3 => :hit_point, :x4 => :collision, :x5 => :eb_intervention)
rename!(df, :x6 => :dv_collision, :x7 => :v_mean, :x8 => :a_mean, :x9 => :a_jerk, :x10 => :a_min)
CSV.write("results_EmergancyBrakingSystem_no.csv", df);


(sum_collisions, sum_eb, dv, v_mean, a_mean, a_jerk, a_min) = EmergencyBrakingSystem.evaluateScenariosMetric(results)
println("#collisions: ", sum_collisions, " #emergency brake interventions: ", sum_eb, " dv_m: ", dv, " v_mean: ", v_mean, " a_mean: ", a_mean, " a_jerk: ", a_jerk, " a_min: ", a_min)

df
